In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
from pytube import YouTube
import pytube
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import math
import cv2
import os
import unicodedata
import string
import time
from functools import wraps
from colorama import Fore

In [2]:
valid_filename_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
char_limit = 255

def clean_filename(filename, whitelist=valid_filename_chars, replace=' '):
    # replace spaces
    for r in replace:
        filename = filename.replace(r,'_')
    
    # keep only valid ascii chars
    cleaned_filename = unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore').decode()
    
    # keep only whitelisted chars
    cleaned_filename = ''.join(c for c in cleaned_filename if c in whitelist)
    if len(cleaned_filename)>char_limit:
        print("Warning, filename truncated because it was over {}. Filenames may no longer be unique".format(char_limit))
    return cleaned_filename[:char_limit]    

In [3]:
def retry(ExceptionToCheck, tries=4, delay=3, backoff=2, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    http://www.saltycrane.com/blog/2009/11/trying-out-retry-decorator-python/
    original from: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry

    :param ExceptionToCheck: the exception to check. may be a tuple of
        exceptions to check
    :type ExceptionToCheck: Exception or tuple
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck:
                    msg = "%s, Retrying in %d seconds..." % (str(ExceptionToCheck), mdelay)
                    if logger:
                        #logger.exception(msg) # would print stack trace
                        logger.warning(msg)
                    else:
                        print(msg)
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry  

In [ ]:
# Store data in lists

pages = [str(i) for i in range(1,202000,250)]
count = 0

#Prepare monitoring of loop
requests = 0
#for every page
for page in pages:
    #initially write the files
    plotExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\movie_plots.txt')
    nameExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\movie_names.txt')
#     errorExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\movie_errors.txt')
    requestExists= os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\requests.txt')
    boxOfficeExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\box_office.txt')
    owBoxOfficeExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\ow_box_office.txt')
    runtimeExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\movie_runtime.txt')
    castExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\movie_cast.txt')
    productionCoExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\productionCo.txt')
    directorsExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\directors.txt')
    effectsExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\specialEffectsCo.txt')
    screenWritersExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\screenWriters.txt')
    budgetExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\movie_budget.txt')
    metascoreExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\metascore.txt')
    releaseExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\releasedate.txt')
    genreExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\movie_genre.txt')
    mpaaExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\movie_mpaa.txt')
    imdbExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\imdb_rating.txt')
    reviewsExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\test\movie_reviews.txt')
    if not plotExists:
        moviePlot = open("plot_sums/test/movie_plots.txt", "w")
        moviePlot.close()
    if  not nameExists:
        movieName = open("plot_sums/test/movie_names.txt", "w")
        movieName.close()
    if not requestExists:
        movieRequest = open("plot_sums/test/movie_request.txt", "w")
        movieRequest.close()
    if not boxOfficeExists:
        boxOffice = open("plot_sums/test/box_office.txt", "w")
        boxOffice.close()
    if not owBoxOfficeExists:
        owBoxOffice = open("plot_sums/test/ow_box_office.txt", "w")
        owBoxOffice.close()
    if not runtimeExists:
        movieRuntime = open("plot_sums/test/movie_runtime.txt", "w")
        movieRuntime.close()
    if not castExists:
        movieCast = open("plot_sums/test/movie_cast.txt", "w")
        movieCast.close()
    if not productionCoExists:
        productionCo = open("plot_sums/test/productionCo.txt", "w")
        productionCo.close()
    if not directorsExists:
        movieDirectors = open("plot_sums/test/directors.txt", "w")
        movieDirectors.close()
    if not effectsExists:
        specialEffectsCo = open("plot_sums/test/specialEffectsCo.txt", "w")
        specialEffectsCo.close()
    if not screenWritersExists:
        screenWriters = open("plot_sums/test/screenWriters.txt", "w")
        screenWriters.close()
    if not budgetExists:
        movieBudget = open("plot_sums/test/screenWriters.txt", "w")
        movieBudget.close()
    if not metascoreExists:
        movieMetascore = open("plot_sums/test/metascore.txt", "w")
        movieMetascore.close()
    if not releaseExists:
        movieRelease = open("plot_sums/test/releasedate.txt", "w")
        movieRelease.close()
    if not genreExists:
        movieGenre = open("plot_sums/test/movie_genre.txt", "w")
        movieGenre.close()
    if not mpaaExists:
        movieMpaa = open("plot_sums/test/movie_mpaa.txt", "w")
        movieMpaa.close()
    if not imdbExists:
        imdbRating = open("plot_sums/test/movie_mpaa.txt", "w")
        imdbRating.close()
    if not reviewsExists:
        movieReviews = open("plot_sums/test/movie_reviews.txt", "w")
        movieReviews.close()
    if(int(page)>1):
        movieRequest = open("plot_sums/movie_request.txt", "a")
        movieRequest.write('[Count #: {}]'.format(count))
        movieRequest.write("\n")
        movieRequest.write("[Current URL: https://www.imdb.com{}]".format(linkzor))
        movieRequest.close()
    
    #make get request
    if int(page) < 10000:
        @retry(ConnectionResetError, tries=20, delay=2,backoff=2)
        def get_response():
            response = get("https://www.imdb.com/search/title?title_type=feature,tv_movie&languages=en&sort=boxoffice_gross_us,desc&count=250&start=" + page)
            return response
        response = get_response()

        
    #monitor requests
    requests += 1
    print('Request:{};'.format(requests))
    clear_output(wait = True)

    # parse the content of request
    page_html = BeautifulSoup(response.text, 'html.parser')

    #select all 250 movie containers from a single page
    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    try:
        linkzor = page_html.find("div", {"id": "main"}).div.div.find('div', class_ = 'desc').find("a", class_ = 'lister-page-next next-page')['href']
        next_page = "https://www.imdb.com{}".format(linkzor)
        movieRequest = open("plot_sums/test/movie_request.txt", "a")
        movieRequest.write('[Request #: {}]'.format(requests + 1))
        movieRequest.write("\n")
        movieRequest.write("[Next Link:{}]".format(linkzor))
        movieRequest.write("\n")
        movieRequest.close()
    except:
        movieRequest = open("plot_sums/movie_request.txt", "a")
        movieRequest.write("Check NEXT button for: {}".format(response.url))
        movieRequest.write("\n")
        movieRequest.close()
        break
    if int(page) >= 10000:
        @retry(ConnectionResetError, tries=20, delay=2,backoff=2)
        def get_response():
            response = get(next_page)
            return response
        response = get_response()

    # Extract data from indiv. movie containers
    for container in mv_containers:
        count+=1
        profile = container.h3.a['href']
        @retry(ConnectionResetError, tries=20, delay=2, backoff=2)
        def get_details():
            details = get("https://www.imdb.com" + profile)
            return details
        details = get_details()
        # parse the content of request
        details_html = BeautifulSoup(details.text, 'html.parser')
        
        '''
            GRAB MOVIE NAME
        '''
        name = container.h3.a.text

        movieName = open("plot_sums/test/movie_names.txt", "a")
        movieName.write(name)
        movieName.write(",,,")
        movieName.close()
        
        '''
            GET METASCORE
        '''
        
        try:
            metascore = container.find('span', class_ = 'metascore')
            metascore = metascore.text
            movieMetascore = open("plot_sums/test/metascore.txt", "a")
            movieMetascore.write(metascore)
            movieMetascore.write(",,,")
            movieMetascore.close()
            print(Fore.GREEN + 'Metascore: {}'.format(metascore))
        except:
            movieMetascore = open("plot_sums/test/metascore.txt", "a")
            movieMetascore.write("null")
            movieMetascore.write(",,,")
            movieMetascore.close()
            print(Fore.RED + 'Metascore')

            
        '''
            GET IMDB RATING
        '''
            
        try:
            imdb = container.strong.text
            imdbRating = open("plot_sums/test/imdb_rating.txt", "a")
            imdbRating.write(imdb)
            imdbRating.write(",,,")
            imdbRating.close()
            print(Fore.GREEN + 'IMDb Rating: {}'.format(imdb))
        except:
            imdbRating = open("plot_sums/test/imdb_rating.txt", "a")
            imdbRating.write("null")
            imdbRating.write(",,,")
            imdbRating.close()
            print(Fore.RED + 'IMDb Rating')
            
        '''
            GRAB PLOT SYNOPSIS
        '''
        try:
            synopsis_link = profile
            @retry(ConnectionResetError, tries=20, delay=2, backoff=2)
            def get_synopsis():
                synopsis = get("https://www.imdb.com/" + synopsis_link + "plotsummary")
                return synopsis
            synopsis = get_synopsis()
            synopsis_html = BeautifulSoup(synopsis.text, 'html.parser')

            plot_synopsis_content = synopsis_html.find("ul", {"id": "plot-synopsis-content"}).li.text
            plot_synopsis_content = plot_synopsis_content.strip()
            if plot_synopsis_content[:58] != "It looks like we don't have a Synopsis for this title yet.":
                moviePlot = open("plot_sums/test/movie_plots.txt", "a")
                moviePlot.write(plot_synopsis_content)
                moviePlot.write(",,,")
                moviePlot.write("\n")
                moviePlot.close()
                print('Request:{}; Movie:{} - {}'.format(requests, count, name))
                print(Fore.GREEN + 'Plot Synopsis')
            else:
                moviePlot = open("plot_sums/test/movie_plots.txt", "a")
                moviePlot.write("null")
                moviePlot.write(",,,")
                moviePlot.write("\n")
                moviePlot.close()
                print('Request:{}; Movie:{} - {}'.format(requests, count, name))
                print(Fore.RED + 'Plot Synopsis')
        except:
            moviePlot = open("plot_sums/test/movie_plots.txt", "a")
            moviePlot.write("null")
            moviePlot.write(",,,")
            moviePlot.write("\n")
            moviePlot.close()
            print('Request:{}; Movie:{} - {}'.format(requests, count, name))
            print(Fore.RED + 'Plot Synopsis')

        '''
            GRAB BOX OFFICE GROSS
        '''
        try:
            #Gross
            bo = details_html.find(text='Gross USA:').parent.findNext('span').decompose()
            bo_gross = details_html.find(text='Gross USA:').parent.parent.text.strip()
            bo_gross = bo_gross[10:].replace('$', '').replace(',', '').strip()
            boxOffice = open("plot_sums/test/box_office.txt", "a")
            boxOffice.write(bo_gross)
            boxOffice.write(",,,")
            boxOffice.close()
            print(Fore.GREEN + 'Box Office Gross: {}'.format(bo_gross))
        except:
            boxOffice = open("plot_sums/test/box_office.txt", "a")
            boxOffice.write("null")
            boxOffice.write(",,,")
            boxOffice.close()
            print(Fore.RED + 'Box Office Gross: Null')
            
        '''
            GRAB OPENING WEEKEND BOX OFFICE
        '''
        try:
            #opening weekend BO
            bo = details_html.find(text='Opening Weekend USA:').parent.findNext('span').decompose()
            bo = details_html.find(text='Opening Weekend USA:').parent.findNext('span').decompose()
            ow_bo = details_html.find(text='Opening Weekend USA:').parent.parent.text.strip()
            ow_bo = ow_bo[20:].replace('$', '').replace(',', '').strip()
            owBoxOffice = open("plot_sums/test/ow_box_office.txt", "a")
            owBoxOffice.write(ow_bo)
            owBoxOffice.write(",,,")
            owBoxOffice.close()
            print(Fore.GREEN + 'Opening Weekend B.O. : {}'.format(ow_bo))
        except:
            owBoxOffice = open("plot_sums/test/ow_box_office.txt", "a")
            owBoxOffice.write("null")
            owBoxOffice.write(",,,")
            owBoxOffice.close()
            print(Fore.RED + 'Opening Weekend B.O. : Null')

        '''
            GRAB RUNTIME
        '''
        
        try:
            runtime = container.find('span', class_ = 'runtime').text
            movieRuntime = open("plot_sums/test/movie_runtime.txt", "a")
            movieRuntime.write(runtime)
            movieRuntime.write(",,,")
            movieRuntime.close()
            print(Fore.GREEN + 'Runtime : {}'.format(runtime))
        except:
            movieRuntime = open("plot_sums/test/movie_runtime.txt", "a")
            movieRuntime.write("null")
            movieRuntime.write(",,,")
            movieRuntime.close()
            print(Fore.GREEN + 'Runtime : Null')
        
        '''
            GRAB CAST
        '''
        @retry(ConnectionResetError, tries=20, delay=2, backoff=2)
        def get_cast():
            cast = get("https://www.imdb.com/" + profile + "fullcredits")
            return cast
        cast = get_cast()
        cast_html = BeautifulSoup(cast.text, 'html.parser')
        
        try:    
            #cast
            cast_html.find('table', class_ = 'cast_list').findNext('tr').decompose()
            cast_members = []
            cast_odd = cast_html.find('table', class_ = 'cast_list').findAll('tr', class_='odd')
            cast_even = cast_html.find('table', class_ = 'cast_list').findAll('tr', class_='even')
            for cast in cast_odd:
                cast_members.append(cast.findAll('td')[1].text.strip())
            for cast in cast_even:
                cast_members.append(cast.findAll('td')[1].text.strip())
            movieCast = open("plot_sums/test/movie_cast.txt", "a")
            for member in cast_members:
                movieCast.write(member)
                movieCast.write(",")
            movieCast.write(",,")
            movieCast.close()
            print(Fore.GREEN + 'Cast')

        except:
            movieCast = open("plot_sums/test/movie_cast.txt", "a")
            movieCast.write("null")
            movieCast.write(",,,")
            movieCast.close()
            print(Fore.RED + 'Cast')


        '''
            GET PRODUCTION COMPANIES
        '''
        
        @retry(ConnectionResetError, tries=20, delay=2, backoff=2)
        def get_production():
            production = get("https://www.imdb.com/" + profile + "companycredits")
            return production
        production = get_production()
        production_html = BeautifulSoup(production.text, 'html.parser')
        try:
            production_comps = []
            comps_x = production_html.find("div", {"id": "company_credits_content"}).ul.findAll('a')
            productionCo = open("plot_sums/test/productionCo.txt", "a")
            for comp in comps_x:
                productionCo.write(comp.text)
                productionCo.write(",")
            productionCo.write(",,")
            productionCo.close()
            print(Fore.GREEN + 'Production COs')
        except:
            productionCo = open("plot_sums/test/mproductionCo.txt", "a")
            productionCo.write("null")
            productionCo.write(",,,")
            productionCo.close()
            print(Fore.RED + 'Production COs')

            
        '''
            GET DIRECTORS
        '''
        
        #Director
        try:
            director = container.findAll('div')[3].findAll('p')[2].findAll('a')[0].text
            movieDirectors = open("plot_sums/test/directors.txt", "a")
            movieDirectors.write(director)
            movieDirectors.write(",,,")
            movieDirectors.close()
            print(Fore.GREEN + 'Directors')
        except:
            movieDirectors = open("plot_sums/test/directors.txt", "a")
            movieDirectors.write("null")
            movieDirectors.write(",,,")
            movieDirectors.close()
            print(Fore.RED + 'Directors')

        '''
            GET SCREEN WRITERS
        '''
        try:
            credit_containers = cast_html.find_all('table', class_ = 'simpleTable simpleCreditsTable')
            writers_x = credit_containers[1].tbody.findAll('tr')
            screenWriters = open("plot_sums/test/screenWriters.txt", "a")
            for writer in writers_x:
                if writer.find('td', attrs = {'colspan':'3'}):
                    1==1
                else:
                    answer = writer.find('td', class_ = 'name').text
                    screenWriters.write(answer.strip())
                    screenWriters.write(",")
            screenWriters.write(",,,")
            screenWriters.close()
            print(Fore.GREEN + 'Screen Writers')
        except:
            screenWriters = open("plot_sums/test/screenWriters.txt", "a")
            screenWriters.write("null,,,")
            screenWriters.close()
            print(Fore.RED + 'Screen Writers')
            
        '''
            GET BUDGET
        '''
        
        try:
            bo_budget = details_html.find(text='Budget:').parent.findNext('span').decompose()
            budget = details_html.find(text='Budget:').parent.parent.text.strip()
            budget = budget[7:].replace('$', '').replace(',', '')
            movieBudget = open("plot_sums/test/movieBudget.txt", "a")
            movieBudget.write(budget)
            movieBudget.write(",,,")
            movieBudget.close()
            print(Fore.GREEN + 'Budget: {}'.format(budget))
        except:
            movieBudget = open("plot_sums/test/movieBudget.txt", "a")
            movieBudget.write("null")
            movieBudget.write(",,,")
            movieBudget.close()
            print(Fore.RED + 'Budget')
            

            
        '''
            GET RELEASE DATES
        '''
        try:
            details_html.find(text='Release Date:').parent.findNext('span').decompose()
            date = details_html.find(text='Release Date:').parent.parent.text.strip()
            date = re.sub(r'\([^)]*\)', '', date[14:])[:-1]
            movieRelease = open("plot_sums/test/releasedate.txt", "a")
            movieRelease.write(date)
            movieRelease.write(",,,")
            movieRelease.close()
            print(Fore.GREEN + 'Release: {}'.format(date))
        except:
            movieRelease = open("plot_sums/test/releasedate.txt", "a")
            movieRelease.write("null")
            movieRelease.write(",,,")
            movieRelease.close()
            print(Fore.RED + 'Release')

            
        '''
            GET GENRE
        '''
        try:
            genre = container.p.find('span', class_ = 'genre').text.strip()
            movieGenre = open("plot_sums/test/movie_genre.txt", "a")
            movieGenre.write(genre)
            movieGenre.write(",,,")
            movieGenre.close()
            print(Fore.GREEN + 'Genre: {}'.format(genre))
        except:
            movieGenre = open("plot_sums/test/movie_genre.txt", "a")
            movieGenre.write("null")
            movieGenre.write(",,,")
            movieGenre.close()
            print(Fore.RED + 'Genre')
            
        '''
            GET SPECIAL EFFECT COMPANIES
        '''
        
        try:
            comps_x = production_html.find("h4", {"id": "specialEffects"}).findNext('ul').findAll('li')
            specialEffectsCo = open("plot_sums/test/specialEffectsCo.txt", "a")
            for comp in comps_x:
                specialEffectsCo.write(comp.a.text)
                specialEffectsCo.write(",")
            specialEffectsCo.write(",,")
            specialEffectsCo.close()
            print(Fore.GREEN + 'Special Effects COs')
        except:
            specialEffectsCo = open("plot_sums/test/specialEffectsCo.txt", "a")
            specialEffectsCo.write("null,,,")
            specialEffectsCo.close()
            
        '''
            GET MPAA RATING
        '''
        try:
            mpaa = container.p.find('span', class_ = 'certificate').text
            movieMpaa = open("plot_sums/test/movie_mpaa.txt", "a")
            movieMpaa.write(mpaa)
            movieMpaa.write(',,,')
            movieMpaa.close()
            print(Fore.GREEN + 'MPAA: {}'.format(mpaa))
        except:
            movieMpaa = open("plot_sums/test/movie_mpaa.txt", "a")
            movieMpaa.write("null")
            movieMpaa.write(',,,')
            movieMpaa.close()
            print(Fore.RED + 'MPAA')

        '''
            GET REVIEWS
        '''
        comments = []
        reviews_link = profile


        @retry(ConnectionResetError, tries=20, delay=2, backoff=2)
        def get_reviews():
            all_reviews = get("https://www.imdb.com/" + reviews_link + "reviews")
            return all_reviews
        all_reviews = get_reviews()
        html_reviews = BeautifulSoup(all_reviews.text, 'html.parser')
        num_reviews = int(html_reviews.find("div", {"id": "main"}).section.find('div', class_ = 'lister').span.text[:-7].strip().replace(',', ''))
        math.ceil(num_reviews/25) -1


        driver = webdriver.Chrome("C:\\Users\Brian\Downloads\chromedriver\chromedriver.exe")
        @retry(ConnectionResetError, tries=20, delay=2, backoff=2)
        def driver_reviews():
            driver.get("https://www.imdb.com/" + reviews_link + "reviews")
        driver_reviews()
        try:
            button = driver.find_element_by_id('load-more-trigger')
            for i in range(math.ceil(num_reviews/25) - 1):
                button.click()
                sleep(randint(1,1))
        except:
            print(Fore.RED + '{} only has 1 page of movie reviews'.format(name))
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')
        list_of_attributes = {"class":"lister-item"}
        review_containers = parser.findAll("div", attrs=list_of_attributes)
        for container in review_containers:
            comments.append(container.div.div.find("div", class_ = "content").div.text)
        movieReviews = open("plot_sums/test/movie_reviews.txt", "a", encoding="utf-8")
        for comment in comments:
            movieReviews.write(comment)
            movieReviews.write("######comment######")
        movieReviews.write("######comments######")
        movieReviews.close()
        driver.close()
        print(Fore.GREEN + 'Comments')
        clear_output(wait = True)

Metascore: 81        
IMDb Rating: 8.0
Request:1; Movie:1 - Star Wars: Episode VII - The Force Awakens
Plot Synopsis
Box Office Gross: 936662225
Opening Weekend B.O. : 247966675
Runtime : 136 min
Cast
Production COs
Directors
Screen Writers
Budget: 245000000
Release: 18 December 2015
Genre: Action, Adventure, Fantasy
Special Effects COs
MPAA: PG-13
Star Wars: Episode VII - The Force Awakens only has 1 page of movie reviews
